Week 5, Assignment C1:

Logistic Regression with Yahoo! Finance API in Python

In this notebook, we will study Logistic Regression using the Yahoo! Finance API in Python.

Please complete the lines bellow where you see "#TODO".

In [1]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 6.6 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=b740de006316d74e9394e46af594aba291c908bcbb8e163bc9032f229312817d
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [2]:
#Objective 1.
#Use this cell to import the Numpy (as np), Pandas (as pd), and YFinance (as yf) packages.

import numpy as np
import pandas as pd
import yfinance as yf
import datetime

#Our work will also require some components of the Sklearn and Pandas_Datareader packages as imported below:

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from pandas_datareader import data as pdr
yf.pdr_override()

In [3]:
#Objective 2.
#Select a stock symbol for a stock whose historical data is available on the Yahoo! Finance website.  Store the
#string of your chosen symbol to the new variable "stock_symbol".

stock_symbol = "TSLA"

In [4]:
#Objective 3.
#Use the function "pdr.get_data_yahoo(stock_symbol, start_date, end_date)" to generate a Pandas dataframe of
#historical stock data for your chosen stock.  Retain only the first four columns of the dataframe.  Be sure to 
#drop any rows containing NaN's, and take a peek at the resulting dataframe to make sure everything looks good.

df = pdr.get_data_yahoo(stock_symbol, "2018-1-1", "2021-1-1")
df=  df[df.columns[0:4]]
df = df[["Open", "High", "Low", "Close"]]
df = df.dropna()
df 



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close
Date,,,,
2018-01-02,62.400002,64.421997,62.200001,64.106003
2018-01-03,64.199997,65.050003,63.110001,63.450001
2018-01-04,62.574001,63.709999,61.136002,62.924000
2018-01-05,63.324001,63.448002,62.400002,63.316002
2018-01-08,63.200001,67.403999,63.099998,67.281998
...,...,...,...,...
2020-12-24,642.989990,666.090027,641.000000,661.770020
2020-12-28,674.510010,681.400024,660.799988,663.690002
2020-12-29,661.000000,669.900024,655.000000,665.989990


In [5]:
#Objective 4.
#Use the historical data to define some predictor variables.  Add these variables to the dataframe.  Include, at a
#minimum, (Predictor.I) the rolling average closing price over the past fifteen (15) days and (Predictor.II) the
#change in opening price over the past one (1) day.  Be sure to again drop any rows containing NaN's, and take a
#peek at the resulting dataframe to make sure everything looks good.  Store the dataframe as the new variable "X".

df['Close_15_Rolling'] = df["Close"].rolling(window=15).mean()
df['Open_1_Change'] = df["Open"].diff()
df = df.dropna()
X = df.copy()
X

,Open,High,Low,Close,Close_15_Rolling,Open_1_Change
Date,,,,,,
2018-01-23,72.000000,72.099998,70.199997,70.557999,67.122800,2.120003
2018-01-24,70.916000,70.949997,68.704002,69.178001,67.460933,-1.084000
2018-01-25,69.653999,69.839996,67.279999,67.528000,67.732800,-1.262001
2018-01-26,68.300003,68.800003,67.141998,68.570000,68.109200,-1.353996
2018-01-29,67.970001,70.169998,67.655998,69.905998,68.548533,-0.330002
...,...,...,...,...,...,...
2020-12-24,642.989990,666.090027,641.000000,661.770020,638.461336,10.789978
2020-12-28,674.510010,681.400024,660.799988,663.690002,642.771338,31.520020
2020-12-29,661.000000,669.900024,655.000000,665.989990,644.386670,-13.510010


In [6]:
#Objective 5.
#Define the target or dependent variable to be one (1) if the change in closing price over the past one (1) day is
#nonnegative and negative one (-1) if the change in closing price over the past one (1) day is negative.  This
#variable's values should be forward-looking (i.e., you should subtract today's price from tomorrow's price rather
#than subtracting yesterday's price from today's price).  Store the resulting values as the new variable "y".  You
#may find the "np.where(*args)" function to be useful.  Look it up in Numpy documentation for support.

y = np.where(-df["Close"].diff(-1) > 0, 1, -1)
y

array([-1, -1,  1,  1, -1,  1, -1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,
        1, -1, -1,  1,  1,  1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,
       -1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,  1, -1,  1,  1,  1, -1,
       -1,  1, -1, -1,  1, -1, -1,  1,  1, -1, -1,  1, -1,  1,  1, -1,  1,
        1, -1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1,  1, -1,
        1,  1,  1, -1,  1,  1, -1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1,
        1, -1, -1, -1,  1,  1,  1, -1, -1, -1, -1, -1,  1,  1, -1, -1,  1,
       -1,  1,  1, -1, -1, -1, -1,  1, -1, -1, -1,  1,  1,  1, -1, -1,  1,
       -1, -1,  1,  1, -1, -1, -1, -1,  1,  1, -1, -1,  1, -1, -1, -1, -1,
       -1, -1, -1,  1, -1,  1, -1,  1, -1,  1, -1, -1,  1, -1,  1,  1,  1,
        1, -1, -1,  1, -1, -1, -1, -1, -1,  1, -1, -1,  1,  1,  1, -1, -1,
       -1,  1,  1, -1,  1,  1,  1, -1,  1,  1,  1, -1, -1,  1,  1, -1, -1,
        1,  1,  1,  1, -1, -1, -1, -1,  1, -1,  1, -1,  1,  1,  1,  1, -1,
        1,  1, -1,  1, -1

In [7]:
#Objective 6.
#Split the data into training and test sets, putting the first seventy percent (70%) of the data in the training
#set.

index = int(0.7*len(X))
X_train, X_test, y_train, y_test = X[:index], X[index:], y[:index], y[index:]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(519, 6) (519,) (223, 6) (223,)


In [8]:
#Objective 7.
#Instantiate the Logistic Regression model object, and use its ".fit(*args)" method to fit the model to the 
#training data.

logistic = LogisticRegression()
logistic = logistic.fit(X_train,y_train)

In [9]:
#Objective 8.
#Examine the model's coefficients by using its ".coef_" method.

pd.DataFrame(zip(X.columns, np.transpose(logistic.coef_)))

,0,1
0,Open,[0.10637948183674158]
1,High,[-0.11225256427394267]
2,Low,[-0.036234778220713436]
3,Close,[0.06341039711734278]
4,Close_15_Rolling,[-0.016906830072924296]
5,Open_1_Change,[-0.012352282220704437]


In [10]:
#Objective 9.
#Use the model's ".predict_proba(*args)" and ".predict(*args)" methods to generate predictions over the test set.

probabilities = logistic.predict_proba(X_test)
predictions = logistic.predict(X_test)

In [11]:
#Objective 10.
#Use the function "metrics.confusion_matrix(*args)" to create a confusion matrix comparing the predicted and true
#classification labels over the test set.

metrics.confusion_matrix(y_test, predictions)

array([[ 13,  84],
       [ 22, 104]])

In [12]:
#Objective 11.
#Calculate the model's accuracy on the test set using its ".score(*args)" method.

logistic.score(X_test, y_test)

0.5246636771300448

In [14]:
#Objective 12.
#Use five-fold cross validation to cross-check the accuracy of the model over different held-out test sets.  This
#is where you should use the function "cross_val_score(*args)".

cross_val= cross_val_score(LogisticRegression(), X, y, cv=5)
cross_val

array([0.41610738, 0.48993289, 0.47297297, 0.46621622, 0.51351351])